In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import ResNet50
from glob import glob as gl
import cv2
import numpy as np
from collections import deque
from timeit import default_timer as timer

In [4]:
resnet_backbone = ResNet50(weights="imagenet", include_top=False, pooling=None, input_tensor=Input(shape=(224, 224, 3)))
backbone_output = resnet_backbone.output

# head
head = AveragePooling2D(pool_size=(7, 7))(backbone_output)
head = Flatten(name="flatten")(head)
head = Dense(512, activation="relu")(head)
head = Dropout(0.5)(head)
head = Dense(512, activation="relu")(head)
head = Dropout(0.25)(head)
head = Dense(256, activation="relu")(head)
head = Dropout(0.25)(head)
head = Dense(64, activation="relu")(head)
head = Dropout(0.25)(head)
head = Dense(1, activation="sigmoid")(head)

model = Model(inputs=resnet_backbone.input, outputs=head)
model.load_weights('ep003-loss0.549-val_loss0.523.h5')

In [5]:
model.save('full_model.h5')

In [3]:
RAW_VIDEO_DATA_DIR = 'data/raw_video/other' # @video folder here
RAW_VIDEO_DATA_PATH = gl(RAW_VIDEO_DATA_DIR + '/*')
labels = ['other', 'wood-making']

In [8]:
def get_frame(cap, sec, frame_index):
    cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
    success, image = cap.read()

    return success, image

In [16]:
for vid_path in RAW_VIDEO_DATA_PATH:
    cap = cv2.VideoCapture('Making the _Impossible Joint_ 四方鎌継.mp4')
    vid_name = vid_path.split('/')[-1]
    Q = []

    if (cap.isOpened()== False): 
      print("Error opening video stream or file")

    frame_index = 0
    frame_rate = 2 # predict every 2 second
    sec = 0
    success, image = get_frame(cap, sec, frame_index)
    while success:
        
        image = cv2.resize(image, (224, 224))
        image = np.expand_dims(image, axis=0)
        preds = int(model.predict(image)[0][0])
        Q.append(preds)
        
        frame_index += 1
        sec = sec + frame_rate
        sec = round(sec, 2)
        success, image = get_frame(cap, sec, frame_index)

    counts = np.bincount(np.array(Q))
    output_label = 100*np.true_divide(counts, len(Q))
    
    print('Video {} has {}% content is other and {}% content is wood-making'.format(vid_name, output_label[0], output_label[1]))
    cap.release()

Video data/raw_video/other\1.3M Views -Supporting tools and creative ideas!.mp4 has 1.5086206896551724% content is other and 98.49137931034483% content is wood-making
